# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import warnings
warnings.filterwarnings('ignore')

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
# Load the csv from WeatherPy
cities_csv = "output_data/WeatherPy/cities.csv"

cities_weather = pd.read_csv(cities_csv, index_col=False)
cities_weather

,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,0,Ilulissat,100,GL,1665622909,64,69.2167,-51.1000,28.42,5.75
1,1,Albany,100,US,1665622909,67,42.6001,-73.9662,61.21,4.00
2,2,Mataura,3,NZ,1665622909,43,-46.1927,168.8643,56.73,3.69
3,3,Avera,100,US,1665622910,98,33.1940,-82.5271,68.38,12.97
4,4,Nikolskoye,66,RU,1665622910,83,59.7035,30.7861,39.09,2.64
...,...,...,...,...,...,...,...,...,...,...
496,496,Preobrazheniye,100,RU,1665623080,61,42.9019,133.9064,56.91,7.09
497,497,San Cristobal,97,VE,1665623080,95,7.7669,-72.2250,69.98,2.35
498,498,Linjiang,100,CN,1665623081,62,41.8069,126.9078,54.79,2.75
499,499,Nuuk,40,GL,1665622836,86,64.1835,-51.7216,29.01,14.97


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Configure gmaps with API key
gmaps.configure(api_key = g_key)

In [4]:
# Store 'Lat' and 'Lng' into  locations 
locations = cities_weather[["Lat", "Lng"]].astype(float)

# Store 'Humidity' into weights
weights = cities_weather["Humidity"].astype(float)

In [5]:
# Create a humidity Heatmap layer
fig = gmaps.figure(center = [0,0] ,zoom_level = 2)

heat_layer = gmaps.heatmap_layer(locations, weights=weights, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 2)

fig.add_layer(heat_layer)

# Display map
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [12]:
# Create a new dataframe with the cities filtered for the following conditions:
# Max temp inbetween 70 and 80F, Windspeed less than 10 mph and 0% Cloudiness.
cities_weather_fit = cities_weather[(cities_weather["Max Temp"] < 80) & (cities_weather["Max Temp"] > 70)
                                                                      & (cities_weather["Wind Speed"] < 10)
                                                                      & (cities_weather["Cloudiness"] == 0) ]
cities_weather_fit

,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
118,118,Angoche,0,MZ,1665622950,91,-16.2325,39.9086,71.02,3.98
125,125,Andrews,0,US,1665622951,41,32.3187,-102.5457,74.28,3.44
155,155,Le Port,0,RE,1665622845,83,-20.9373,55.2919,72.90,5.28
231,231,Klamath Falls,0,US,1665622993,20,42.2249,-121.7817,76.91,0.00
236,236,Prescott,0,US,1665622765,32,34.5400,-112.4685,72.30,5.59
238,238,Tabas,0,IR,1665622995,16,33.5959,56.9244,70.14,3.02
266,266,Saint-Pierre,0,RE,1665622812,68,-21.3393,55.4781,72.57,6.91
281,281,Mitsamiouli,0,KM,1665623012,74,-11.3847,43.2844,74.37,3.89
381,381,Lompoc,0,US,1665623046,85,34.6391,-120.4579,72.79,8.05
419,419,Riyadh,0,SA,1665622513,29,24.6877,46.7219,71.74,1.14


In [13]:
# There are more than 10 cities identified and this can be further narrowed down by filtering out the cities with
# humidity greater than 85%

cities_weather_filtered = cities_weather_fit[(cities_weather_fit["Humidity"] <= 84)]
cities_weather_filtered

,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
125,125,Andrews,0,US,1665622951,41,32.3187,-102.5457,74.28,3.44
155,155,Le Port,0,RE,1665622845,83,-20.9373,55.2919,72.90,5.28
231,231,Klamath Falls,0,US,1665622993,20,42.2249,-121.7817,76.91,0.00
236,236,Prescott,0,US,1665622765,32,34.5400,-112.4685,72.30,5.59
238,238,Tabas,0,IR,1665622995,16,33.5959,56.9244,70.14,3.02
266,266,Saint-Pierre,0,RE,1665622812,68,-21.3393,55.4781,72.57,6.91
281,281,Mitsamiouli,0,KM,1665623012,74,-11.3847,43.2844,74.37,3.89
419,419,Riyadh,0,SA,1665622513,29,24.6877,46.7219,71.74,1.14
459,459,Meadow Lake,0,US,1665623067,40,34.8014,-106.5436,73.06,1.32
479,479,Léo,0,BF,1665623076,64,11.1000,-2.1000,76.82,5.48


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [14]:
# Store the filtered cities data into a variable named hotel_df.
hotel_df = cities_weather_filtered
hotel_df

,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
125,125,Andrews,0,US,1665622951,41,32.3187,-102.5457,74.28,3.44
155,155,Le Port,0,RE,1665622845,83,-20.9373,55.2919,72.90,5.28
231,231,Klamath Falls,0,US,1665622993,20,42.2249,-121.7817,76.91,0.00
236,236,Prescott,0,US,1665622765,32,34.5400,-112.4685,72.30,5.59
238,238,Tabas,0,IR,1665622995,16,33.5959,56.9244,70.14,3.02
266,266,Saint-Pierre,0,RE,1665622812,68,-21.3393,55.4781,72.57,6.91
281,281,Mitsamiouli,0,KM,1665623012,74,-11.3847,43.2844,74.37,3.89
419,419,Riyadh,0,SA,1665622513,29,24.6877,46.7219,71.74,1.14
459,459,Meadow Lake,0,US,1665623067,40,34.8014,-106.5436,73.06,1.32
479,479,Léo,0,BF,1665623076,64,11.1000,-2.1000,76.82,5.48


In [15]:
# Add a "Hotel Name" column to the DataFrame.
hotel_df["Hotel Name"] = ""
hotel_df

,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Hotel Name
125,125,Andrews,0,US,1665622951,41,32.3187,-102.5457,74.28,3.44,
155,155,Le Port,0,RE,1665622845,83,-20.9373,55.2919,72.90,5.28,
231,231,Klamath Falls,0,US,1665622993,20,42.2249,-121.7817,76.91,0.00,
236,236,Prescott,0,US,1665622765,32,34.5400,-112.4685,72.30,5.59,
238,238,Tabas,0,IR,1665622995,16,33.5959,56.9244,70.14,3.02,
266,266,Saint-Pierre,0,RE,1665622812,68,-21.3393,55.4781,72.57,6.91,
281,281,Mitsamiouli,0,KM,1665623012,74,-11.3847,43.2844,74.37,3.89,
419,419,Riyadh,0,SA,1665622513,29,24.6877,46.7219,71.74,1.14,
459,459,Meadow Lake,0,US,1665623067,40,34.8014,-106.5436,73.06,1.32,
479,479,Léo,0,BF,1665623076,64,11.1000,-2.1000,76.82,5.48,


In [16]:
# find a hotel within 5000m radius for each city's coordinates
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

# set up the required parameters
params = {
    "radius": 5000,
    "type": "lodging",
    "keyword": "hotel",
    "key": g_key,
}
# use iterrows to iterate through pandas dataframe
for index, row in hotel_df.iterrows():
    
#   get lat and lng from hotel_df
    lat = row["Lat"]
    lng = row["Lng"]
    
#   add location to params dict
    params["location"] = f"{lat},{lng}"
    
#   assemble url and make API request
    response = requests.get(base_url, params=params).json()
    
#   extract results
    results = response["results"]
    
    try:
        print(f"Closest hotel in {row['City']} is {results[0]['name']}.")
        hotel_df.loc[index, "Hotel Name"] = results[0]['name']
       
    except (KeyError, IndexError):
        print(f"No hotel at {row['City']}")
        hotel_df.loc[index, "Hotel Name"] = "NA"
        
    print("------------")


Closest hotel in Andrews is La Quinta Inn & Suites by Wyndham Andrews.
------------
Closest hotel in Le Port is Lodge Roche Tamarin & Spa Village Nature.
------------
Closest hotel in Klamath Falls is Fairfield Inn & Suites by Marriott Klamath Falls.
------------
Closest hotel in Prescott is Hotel St Michael Prescott.
------------
Closest hotel in Tabas is Amir hotel.
------------
Closest hotel in Saint-Pierre is Le Battant Des Lames.
------------
Closest hotel in Mitsamiouli is Al Camar Lodge.
------------
Closest hotel in Riyadh is Le Park Concord Rabwah Riyadh.
------------
No hotel at Meadow Lake
------------
Closest hotel in Léo is HOTEL SISSILIS.
------------
Closest hotel in Luangwa is Justfeli Lodge, Luangwa.
------------


In [17]:
# Display the filtered cities along with the hotel names
hotel_df

,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Hotel Name
125,125,Andrews,0,US,1665622951,41,32.3187,-102.5457,74.28,3.44,La Quinta Inn & Suites by Wyndham Andrews
155,155,Le Port,0,RE,1665622845,83,-20.9373,55.2919,72.90,5.28,Lodge Roche Tamarin & Spa Village Nature
231,231,Klamath Falls,0,US,1665622993,20,42.2249,-121.7817,76.91,0.00,Fairfield Inn & Suites by Marriott Klamath Falls
236,236,Prescott,0,US,1665622765,32,34.5400,-112.4685,72.30,5.59,Hotel St Michael Prescott
238,238,Tabas,0,IR,1665622995,16,33.5959,56.9244,70.14,3.02,Amir hotel
266,266,Saint-Pierre,0,RE,1665622812,68,-21.3393,55.4781,72.57,6.91,Le Battant Des Lames
281,281,Mitsamiouli,0,KM,1665623012,74,-11.3847,43.2844,74.37,3.89,Al Camar Lodge
419,419,Riyadh,0,SA,1665622513,29,24.6877,46.7219,71.74,1.14,Le Park Concord Rabwah Riyadh
459,459,Meadow Lake,0,US,1665623067,40,34.8014,-106.5436,73.06,1.32,NA
479,479,Léo,0,BF,1665623076,64,11.1000,-2.1000,76.82,5.48,HOTEL SISSILIS


In [18]:
# remove cities with no hotels in 5000m radius
hotel_df = hotel_df[hotel_df["Hotel Name"] != "NA"]
hotel_df

,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Hotel Name
125,125,Andrews,0,US,1665622951,41,32.3187,-102.5457,74.28,3.44,La Quinta Inn & Suites by Wyndham Andrews
155,155,Le Port,0,RE,1665622845,83,-20.9373,55.2919,72.90,5.28,Lodge Roche Tamarin & Spa Village Nature
231,231,Klamath Falls,0,US,1665622993,20,42.2249,-121.7817,76.91,0.00,Fairfield Inn & Suites by Marriott Klamath Falls
236,236,Prescott,0,US,1665622765,32,34.5400,-112.4685,72.30,5.59,Hotel St Michael Prescott
238,238,Tabas,0,IR,1665622995,16,33.5959,56.9244,70.14,3.02,Amir hotel
266,266,Saint-Pierre,0,RE,1665622812,68,-21.3393,55.4781,72.57,6.91,Le Battant Des Lames
281,281,Mitsamiouli,0,KM,1665623012,74,-11.3847,43.2844,74.37,3.89,Al Camar Lodge
419,419,Riyadh,0,SA,1665622513,29,24.6877,46.7219,71.74,1.14,Le Park Concord Rabwah Riyadh
479,479,Léo,0,BF,1665623076,64,11.1000,-2.1000,76.82,5.48,HOTEL SISSILIS
480,480,Luangwa,0,ZM,1665623076,42,-15.6167,30.4167,71.73,1.99,"Justfeli Lodge, Luangwa"


In [19]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [20]:
hotel_info

['\n<dl>\n<dt>Name</dt><dd>La Quinta Inn & Suites by Wyndham Andrews</dd>\n<dt>City</dt><dd>Andrews</dd>\n<dt>Country</dt><dd>US</dd>\n</dl>\n',
 '\n<dl>\n<dt>Name</dt><dd>Lodge Roche Tamarin & Spa Village Nature</dd>\n<dt>City</dt><dd>Le Port</dd>\n<dt>Country</dt><dd>RE</dd>\n</dl>\n',
 '\n<dl>\n<dt>Name</dt><dd>Fairfield Inn & Suites by Marriott Klamath Falls</dd>\n<dt>City</dt><dd>Klamath Falls</dd>\n<dt>Country</dt><dd>US</dd>\n</dl>\n',
 '\n<dl>\n<dt>Name</dt><dd>Hotel St Michael Prescott</dd>\n<dt>City</dt><dd>Prescott</dd>\n<dt>Country</dt><dd>US</dd>\n</dl>\n',
 '\n<dl>\n<dt>Name</dt><dd>Amir hotel</dd>\n<dt>City</dt><dd>Tabas</dd>\n<dt>Country</dt><dd>IR</dd>\n</dl>\n',
 '\n<dl>\n<dt>Name</dt><dd>Le Battant Des Lames</dd>\n<dt>City</dt><dd>Saint-Pierre</dd>\n<dt>Country</dt><dd>RE</dd>\n</dl>\n',
 '\n<dl>\n<dt>Name</dt><dd>Al Camar Lodge</dd>\n<dt>City</dt><dd>Mitsamiouli</dd>\n<dt>Country</dt><dd>KM</dd>\n</dl>\n',
 '\n<dl>\n<dt>Name</dt><dd>Le Park Concord Rabwah Riyadh</dd

In [21]:
# Add marker layer on top of the humidity heat map created earlier.
markers = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(markers)

# Display map
fig

Figure(layout=FigureLayout(height='420px'))